# Running suite2p on example data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MouseLand/suite2p/blob/main/notebooks/example_run_suite2p_2026.ipynb)

This notebook will guide you through the various stages and outputs of suite2p by running it on a real-life dataset. This is data collected from a wild-type mouse injected with GCaMP6s in primary visual cortex. The recording was collected at 13Hz (there were 3 planes in the recording, 1 is included here).

The code cell below installs and imports the necessary packages to use suite2p in this Colab notebook.

To use the GPU-acceleration, connect to a hosted runtime with a GPU in colab.

In [ ]:
!pip install suite2p

In [ ]:
import os, requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch

import suite2p

In [ ]:
# Figure Style settings for notebook.
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': True,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

The next code cell downloads the data. You can also upload your own data to this folder on the left in the "Files" menu, or you can connect to your google drive (see instructions [here](https://colab.research.google.com/notebooks/io.ipynb)), which will make it easier to download the output files to your local computer.

In [ ]:
# Run this cell if you'd like to connect/mount your google drive
from google.colab import drive
drive_path = '/content/drive'
drive.mount('/content/drive')

In [ ]:
fname = "gt1.tif"
url = "https://osf.io/download/67f0170c14be54fb766ddbb6"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
        print("Successfully downloaded data !!!")
from tifffile import imread
from pathlib import Path
data = imread(fname)
print('imaging data of shape: ', data.shape)
n_time, Ly, Lx = data.shape

## Set db parameters

db contains configurations related to output directory paths (e.g., `save_path0`: where you want to save suite2p output) and your recording setup (e.g., `nplanes`: number of planes)

In [ ]:
db = {
    "data_path": ['.'], # Directory where your input files are located
    "save_path0": '/content/suite2p_output', # Directory where you want suite2p to write output files.
    "file_list": [fname], # Specify files you'd like to specifically use in the data_path
    "input_format": "tif",
    "nplanes": 1, # each tiff has these many planes in sequence
    "nchannels": 1, # each tiff has these many channels per plane
    "keep_movie_raw": True,
    "batch_size": 200, # we will decrease the batch_size in case low RAM on computer
}
print(db)

## Set pipeline settings

settings will contain the pipeline-specific parameters.

In [ ]:
settings = suite2p.default_settings()
settings['detection']['threshold_scaling'] = 2.0 # we are increasing the threshold for finding ROIs to limit the number of non-cell ROIs found (sometimes useful in gcamp injections)
settings['fs'] = 13 # sampling rate of recording, determines binning for cell detection
settings['tau'] = 1.25 # timescale of gcamp to use for deconvolution
settings['device'] = 'cuda' if torch.cuda.is_available() else 'cpu' # use GPU if available for faster processing
print(settings)

`settings` is a nested dictionary that contains module-specific dictionaries (e.g., registration, detection, classification, extraction).

In [ ]:
print(settings.keys())

## Run entire suite2p pipeline on data
The suite2p.run_s2p function runs the pipeline and returns a list of output dictionaries containing the pipeline parameters used and extra data calculated along the way, one for each plane.

To see the logs during running, you now need to run `logger_setup`, optionally providing the `save_path` for the logs to be written to a text file.

The following cell might take a couple of minutes to run.

In [ ]:
from suite2p.run_s2p import logger_setup
logger_setup()
suite2p.run_s2p(settings=settings, db=db)

### Outputs from the Suite2p Pipeline


#### Results Files
Analysis result files can be found below

In [ ]:
list(Path(db['save_path0']).joinpath('suite2p').iterdir())

## Running individual Suite2P modules

While `suite2p.run_s2p` runs the entire pipeline, you may instead want to run individual modules (e.g., registration, cell detection, extraction, etc.). In this section, we'll go over the steps to run the following individual modules.

*   Registration
*   ROI detection
* Signal Extraction
* Classification of ROIs
* Spike Deconvolution

### Running Registration

To run `registration`, `detection`, and `extraction` separately, we must first talk about a special class in suite2p called a `BinaryFile`. You can think of `BinaryFile` as a class for reading/writing image data that acts like a numpy array.

In [ ]:
from suite2p.io import BinaryFile

# Convert our example tif file into a binary file
if data.dtype == np.uint16:
    data = (data // 2).astype(np.int16)
# Write to binary
data.tofile('raw_data.bin')
f_raw = BinaryFile(Ly=Ly, Lx=Lx, filename='raw_data.bin')
print(f_raw.shape)

# Create a binary file we will write our registered image to
f_reg = suite2p.io.BinaryFile(
    Ly=Ly, Lx=Lx, filename='registered_data.bin',
    n_frames = f_raw.shape[0], write=True
) # Set registered binary file to have same n_frames

We'll run the registration module only on our example image which only contains data from a single channel. You can add in data for the second channel (e.g., `f_reg_chan2` and `f_raw_chan2`) using similar code to what we have above. When writing a new BinaryFile, please make sure to specify the number of frames your `BinaryFile` instance will have. Refer to the docs to see what the outputs refer to.

If you have access to a CUDA-compatible GPU, make sure to set `device=torch.device("cuda")`. Set it to `device=torch.device("cpu")` if you do not.

In [ ]:
settings['device'] = 'cuda' if torch.cuda.is_available() else 'cpu' # use GPU if available for faster processing
device = torch.device(settings['device'])

In [ ]:
from suite2p import registration

reg_outputs = registration.registration_wrapper(
  f_reg, f_raw=f_raw, f_reg_chan2=None, f_raw_chan2=None,
  align_by_chan2=None, save_path= db['save_path0'],
  badframes=None, settings=settings["registration"],
  device=device
)
f_reg.close()


`reg_outputs` will be a dictionary that contains the outputs of registration. Please refer to the docs to see what each key refers to.

In [ ]:
reg_outputs.keys()

### Running Detection

To run ROI detection alone (called by the `detection_wrapper` function in the detection module), we'll first instantiate the necessary parameters. You only need a `BinaryRWFile` corresponding to a registered/unregistered recording. Here, we'll pass the `f_reg` we obtained after running the registration module above.

Suite2p provides a default classification file containing a default dataset that is used to train a classifier that will be used for your data. One could specify their own classification file if they'd like. To do so, they should save a numpy file with a dict containing the following keys:


* `stats`: ROI stats
* `keys`: keys of ROI stats that will be used for classification
* `iscell`: labels specifying whether an ROI is a cell or not


In [ ]:
# Use default classification file provided by suite2p
classfile = suite2p.classification.builtin_classfile
np.load(classfile, allow_pickle=True)[()]

In [ ]:
from suite2p import detection

yrange, xrange = reg_outputs["yrange"], reg_outputs["xrange"]
# Check f_reg before detection
f_reg = BinaryFile(Ly=Ly, Lx=Lx, filename='registered_data.bin')

detect_outputs, stat, redcell = detection.detection_wrapper(
    f_reg, meanImg_chan2=None,
    yrange=yrange, xrange=xrange,
    tau=settings["tau"], fs=settings["fs"],
    diameter=settings["diameter"],
    settings=settings["detection"],
    classifier_path=classfile,
    badframes=None,
    preclassify=settings["classification"]["preclassify"],
    device=device
)

`detect_outputs` will contain configurations that were used for detection.

In [ ]:
detect_outputs.keys()

`stat` will be an array of detected ROIs. `redcell` will be a binary array of shape `num_detected_rois` x 2. The first column will be a binary variable that specifies whether or not the detected ROI is an anatomical channel. The second column will be a measure of the confidence of the algorithm. `redcell` will be `None` if you do not have a second channel (i.e., `meanImg_chan2=None`).

In [ ]:
print(len(stat))
stat[0].keys()

### Running Fluorescence Extraction

To run extraction alone (called by the `extraction_wrapper` function in the extraction module), we can just make use of any `stat` dictionary (from previous runs of suite2p or a custom user-made one). In this case, we'll use the one output by the cell above. If you'd like to extract signal, you can pass a `binaryFile` corresponding to the recording for the second channel to the `f_reg_chan2` parameter.

In [ ]:
from suite2p import extraction

F, Fneu, F_chan2, Fneu_chan2 = extraction.extraction_wrapper(
    stat, f_reg, f_reg_chan2=None, settings=settings["extraction"],
    device=device
)

`F` will be a matrix of shape `num_detected_ROIs` x `n_time`. It contains the calcium signal extracted from each detected ROI. `Fneu` will have the same shape and will contain the background signal detected from the surrounding neuropil. `F_chan2` and `F_neu_chan2` will be similar outputs for the anatomical channel, if provided.

In [ ]:
print(F.shape, Fneu.shape)

### Running Spike Deconvolution

We have to do some preprocessing before running spike deconvolution:


1.   Neuropil subtraction
2.   Baseline Correction

You can read more about these steps in the docs.




In [ ]:
# Neuropil subtraction
dF = F.copy() - settings["extraction"]["neuropil_coefficient"] * Fneu

In [ ]:
# Baseline correction of fluorescence traces
dF = extraction.preprocess(F=dF, fs=settings["fs"], device=device,
                           batch_size=settings["extraction"]["batch_size"],
                           **settings["dcnv_preprocess"])

In [ ]:
spks = extraction.oasis(F=dF, batch_size=settings["extraction"]["batch_size"],
                        tau=settings["tau"], fs=settings["fs"])

### Running Classification of ROIs

Finally, we can use activity-based statistics from the extraction step to classify ROIs as cells or not.

In [ ]:
from suite2p import classification

iscell = classification.classify(stat=stat, classfile=classfile)


`iscell` will be a num_ROIs x 2 array, where the first column is a 1 if it is a cell and 0 if not. The second column is the probability score given by the used classifier.

In [ ]:
iscell

## Visualizations

### Registration

Registration computes a reference image from a subset of frames and registers all frames to the reference.


In [ ]:
plt.subplot(1, 3, 1)

plt.imshow(reg_outputs['refImg'], cmap='gray', )
plt.title("Reference Image for Registration");

plt.subplot(1, 3, 2)
plt.imshow(reg_outputs['meanImg'], cmap='gray')
plt.title("Mean registered image")

plt.subplot(1, 3, 3)
plt.imshow(reg_outputs['meanImgE'], cmap='gray')
plt.title("High-pass filtered Mean registered image");
plt.tight_layout()

The rigid offsets of the frame from the reference are saved in `reg_outputs['yoff']` and `reg_outputs['xoff']`. The nonrigid offsets are saved in `reg_outputs['yoff1']` and `reg_outputs['xoff1']`, and each column is the offsets for a block (default block size will be 128 x 128 pixels).

In [ ]:
block_num = 1 # feel free to vary to see other blocks'
reg_outputs['yoff1'].shape


In [ ]:
plt.figure(figsize=(18,8))

plt.subplot(4,1,1)
plt.plot(reg_outputs['yoff'][:1000])
plt.ylabel('rigid y-offsets')

plt.subplot(4,1,2)
plt.plot(reg_outputs['xoff'][:1000])
plt.ylabel('rigid x-offsets')

plt.subplot(4,1,3)
plt.plot(reg_outputs['yoff1'][:1000, block_num])
plt.ylabel('nonrigid y-offsets')

plt.subplot(4,1,4)
plt.plot(reg_outputs['xoff1'][:1000, block_num])
plt.ylabel('nonrigid x-offsets')
plt.xlabel('frames')

plt.show()

In [ ]:
#@title Run cell to look at registered frames
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from suite2p.io import BinaryFile

widget = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


def plot_frame(t):
    with BinaryFile(Ly=Ly,
                Lx=Lx,
                filename='registered_data.bin') as f:
        plt.imshow(f[t])

interact(plot_frame, t=(0, n_time- 1, 1)); # zero-indexed so have to subtract 1

Here in the notebook is not the best/fastest way to play the movie, you can play it in the suite2p GUI in the "View registered binary" player.

### Detection

ROIs are found by searching for sparse signals that are correlated spatially in the FOV. The ROIs are saved in stat.npy as a list of dictionaries which contain the pixels of the ROI and their weights (`stat['ypix']`, `stat['xpix']`, and `stat['lam']`). It also contains other spatial properties of the ROIs such as their aspect ratio and compactness, and properties of the signal such as the skewness of the fluorescence signal.

In [ ]:
save_path = Path(db['save_path0']).joinpath('suite2p/plane0')
stats_file = save_path.joinpath('stat.npy')
iscell = np.load(save_path.joinpath('iscell.npy'), allow_pickle=True)[:, 0].astype(int)
stats = np.load(stats_file, allow_pickle=True)
print(stats[0].keys())

Some ROIs are defined as "cells" (somatic ROIs) or "not cells" (all other ROIs) depending on their properties, like skewness, compactness, etc. Below we will visualize the ROIs, but please open the files in the suite2p GUI for closer inspection.

In [ ]:
n_cells = len(stats)

h = np.random.rand(n_cells)
hsvs = np.zeros((2, Ly, Lx, 3), dtype=np.float32)

for i, stat in enumerate(stats):
    ypix, xpix, lam = stat['ypix'], stat['xpix'], stat['lam']
    hsvs[iscell[i], ypix, xpix, 0] = h[i]
    hsvs[iscell[i], ypix, xpix, 1] = 1
    hsvs[iscell[i], ypix, xpix, 2] = lam / lam.max()

from colorsys import hsv_to_rgb
rgbs = np.array([hsv_to_rgb(*hsv) for hsv in hsvs.reshape(-1, 3)]).reshape(hsvs.shape)

plt.figure(figsize=(18,18))
plt.subplot(3, 1, 1)
plt.imshow(detect_outputs['max_proj'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.subplot(3, 1, 2)
plt.imshow(rgbs[1])
plt.title("All Cell ROIs")

plt.subplot(3, 1, 3)
plt.imshow(rgbs[0])
plt.title("All non-Cell ROIs");

plt.tight_layout()

### Extraction

We will load in the fluorescence, the neuropil and the deconvolved traces, and visualize them for a few cells.

In [ ]:
f_cells = np.load(save_path.joinpath('F.npy'))
f_neuropils = np.load(save_path.joinpath('Fneu.npy'))
spks = np.load(save_path.joinpath('spks.npy'))
f_cells.shape, f_neuropils.shape, spks.shape

In [ ]:
plt.figure(figsize=[20,20])
plt.suptitle("Fluorescence and Deconvolved Traces for Different ROIs", y=0.92);
rois = np.arange(len(f_cells))[::200]
for i, roi in enumerate(rois):
    plt.subplot(len(rois), 1, i+1, )
    f = f_cells[roi]
    f_neu = f_neuropils[roi]
    sp = spks[roi]
    # Adjust spks range to match range of fluroescence traces
    fmax = np.maximum(f.max(), f_neu.max())
    fmin = np.minimum(f.min(), f_neu.min())
    frange = fmax - fmin
    sp /= sp.max()
    sp *= frange
    plt.plot(f, label="Cell Fluorescence")
    plt.plot(f_neu, label="Neuropil Fluorescence")
    plt.plot(sp + fmin, label="Deconvolved")
    plt.xticks(np.arange(0, f_cells.shape[1], f_cells.shape[1]/10))
    plt.ylabel(f"ROI {roi}", rotation=0)
    plt.xlabel("frame")
    if i == 0:
        plt.legend(bbox_to_anchor=(0.93, 2))